In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import xlrd 

train_df = pd.read_csv('answer_train.csv')
lb = LabelEncoder()
train_df['label'] = lb.fit_transform(train_df['label'])
test_df = pd.read_csv('submit_example_test1.csv')
test_df2 = pd.read_csv('submit_example_test2.csv')

In [3]:
def read_sheet(path):
    workbook = xlrd.open_workbook(path) 
    sheetnames = workbook.sheet_names() 
    content = []
    for sheetN in sheetnames:
        sheet = workbook.sheet_by_name(sheetN) 
        for i in range(sheet.nrows):
            for j in range(sheet.ncols):
                content.append(str(sheet.cell(i,j).value))
    return "".join(content)

In [ ]:
import os
import chardet
from tqdm import tqdm

filename_list = train_df['filename']
content_list = []
error_list = []
for i in tqdm(range(len(filename_list) )):

    bytes = min(32, os.path.getsize(filename_list[i]))
    raw = open(filename_list[i], 'rb').read(bytes)
    result = chardet.detect(raw)
    encoding = result['encoding']
    if encoding in ['ISO-8859-1']:  

        try:
            content = read_sheet(filename_list[i])
        except:
            try:
                new_file_name = filename_list[i].split('.')[0] + '.csv'
                if os.path.exists(new_file_name):
                    new_file_name = new_file_name[:-4] + '9999' + '.csv'
                os.rename(filename_list[i], new_file_name)
                with open(new_file_name, encoding='utf-8') as f:
                    content = f.read()
                os.rename(new_file_name, filename_list[i])
            except:
                error_list.append(i)
                content = ''

    else:
        with open(filename_list[i], encoding='utf-8') as f:
            content = f.read()
    content_list.append(content.replace(',','').replace('\n', ''))       

In [ ]:
pd.DataFrame({
    'id': filename_list,
    'text': content_list,
    'label': train_df['label']
}).to_csv('train_text.csv', index=False)

In [ ]:
filename_list = test_df2['filename']
content_list = []
error_list = []
for i in tqdm(range(len(filename_list) )):

    bytes = min(32, os.path.getsize(filename_list[i]))
    raw = open(filename_list[i], 'rb').read(bytes)
    result = chardet.detect(raw)
    encoding = result['encoding']
    if encoding in ['ISO-8859-1']:  

        try:
            content = read_sheet(filename_list[i])
        except:
            try:
                new_file_name = filename_list[i].split('.')[0] + '.csv'
                if os.path.exists(new_file_name):
                    new_file_name = new_file_name[:-4] + '9999' + '.csv'
                os.rename(filename_list[i], new_file_name)
                with open(new_file_name, encoding='utf-8') as f:
                    content = f.read()
                os.rename(new_file_name, filename_list[i])
            except:
                error_list.append(i)
                content = ''

    else:
        with open(filename_list[i], encoding='utf-8') as f:
            content = f.read()
    content_list.append(content.replace(',','').replace('\n', ''))  

In [6]:
pd.DataFrame({
    'id': filename_list,
    'text': content_list,
}).to_csv('test_text2.csv', index=False)